In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-s0yur5yg/unsloth_416e9ce024f74e4691d79b2178a50668
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-s0yur5yg/unsloth_416e9ce024f74e4691d79b2178a50668
  Resolved https://github.com/unslothai/unsloth.git to commit 8a9e24ed4f092e438bd09436a897f2ca2530e72b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [59]:
!pip install accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!pip install tensorboard

In [1]:
!pip install -U flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.9.post1-cp310-cp310-linux_x86_64.whl size=120889689 sha256=5022ba11d48bf74926da9c16260f4ea2b9bb7f4e29bdb4bd6e1383ad1c55d16f
  Stored in directory: /root/.cache/pip/wheels/cc/ad/f6/7ccf0238790d6346e9fe622923a76ec218e890d356b9a2754a
Successfully built flash-attn


In [1]:
from unsloth import FastLanguageModel
import torch
import os
import json
from datasets import load_dataset, Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import requests
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
import random
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers.integrations import TensorBoardCallback

/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
# from accelerate import Accelerator

In [4]:
# accelerator = Accelerator()

In [5]:
# accelerator = Accelerator(gradient_accumulation_steps=4)
# device = accelerator.device

In [6]:
# Settings
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [7]:
!nvidia-smi

Thu Jun 13 03:13:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8               6W /  60W |     10MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
# torch.cuda.empty_cache()

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto"
)

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.811 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
# !git clone https://github.com/VatsalPatel18/Weather-Chatbot-phi3.git

In [12]:
from datasets import Dataset, DatasetDict
dataset_path = './data/weather_chatbot_dataset1.json'
with open(dataset_path, 'r') as f:
    weather_data = json.load(f)

In [13]:
weather_data[0]

{'user_input': 'What is the current weather in San Jose today night?',
 'intent_extraction': {'intent': 'current_weather',
  'entities': {'city': 'San Jose', 'date': 'today', 'time_of_day': 'night'}},
 'api_response': {'location': 'San Jose, US',
  'temperature': 25.75,
  'description': 'haze',
  'wind_speed': 5.66,
  'humidity': 44},
 'assistant_response': 'The current weather in San Jose night is haze with a temperature of 25.75°C, wind speed of 5.66 meters per second, and humidity of 44%.'}

In [16]:
weather_data[5]

{'user_input': 'What is the current weather in Lagos today?',
 'intent_extraction': {'intent': 'current_weather',
  'entities': {'city': 'Lagos', 'date': 'today'}},
 'api_response': {'location': 'Lagos, NG',
  'temperature': 25.43,
  'description': 'light rain',
  'wind_speed': 2.2,
  'humidity': 91},
 'assistant_response': 'The current weather in Lagos  is light rain with a temperature of 25.43°C, wind speed of 2.2 meters per second, and humidity of 91%.'}

In [19]:
weather_data[11]

{'user_input': 'What is the forecast weather in Frankfurt 2024-06-16 afternoon?',
 'intent_extraction': {'intent': 'forecast_weather',
  'entities': {'city': 'Frankfurt',
   'date': '2024-06-16',
   'time_of_day': 'afternoon'}},
 'api_response': {'location': 'Tokyo, JP',
  'temperature': 22.44,
  'description': 'broken clouds',
  'wind_speed': 1.54,
  'humidity': 85},
 'assistant_response': 'The current weather in Tokyo afternoon is broken clouds with a temperature of 22.44°C, wind speed of 1.54 meters per second, and humidity of 85%.'}

In [29]:
# Split the dataset
intent_identification_data = []
assistant_response_data = []

for entry in weather_data:
    # For intent identification
    intent_identification_data.append({
        "user_input": entry["user_input"],
        "intent_extraction": entry["intent_extraction"]
    })

    # For assistant response generation
    assistant_response_data.append({
        "user_input": entry["user_input"],
        "api_response": entry["api_response"],
        "assistant_response": entry["assistant_response"]
    })

# Combine the datasets
combined_data = intent_identification_data + assistant_response_data

# Shuffle the combined dataset
random.shuffle(combined_data)

# Convert to Hugging Face Dataset format
combined_dataset = Dataset.from_dict({"conversations": combined_data})
dataset_split = combined_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']

In [31]:
train_dataset

Dataset({
    features: ['conversations'],
    num_rows: 1041
})

In [32]:
train_dataset[0]

{'conversations': {'api_response': None,
  'assistant_response': None,
  'intent_extraction': {'entities': {'city': 'San Antonio',
    'date': 'today',
    'time_of_day': 'afternoon'},
   'intent': 'current_weather'},
  'user_input': 'What is the current weather in San Antonio today afternoon?'}}

In [33]:
from unsloth.chat_templates import get_chat_template
from typing import Dict, List

In [36]:
def get_chat_template(tokenizer, chat_template="phi-3", mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt", "API": "api"}):
    return tokenizer

tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt", "API": "api"},
)

def format_combined_prompts(batch: Dict[str, List[Dict]], tokenizer):
    def format_conversation(example: Dict[str, str]):
        conversation = [{"from": "human", "value": example['user_input']}]
        if "intent_extraction" in example and example['intent_extraction'] is not None:
            conversation.append({"from": "gpt", "value": str(example['intent_extraction'])})
        if "api_response" in example and example['api_response'] is not None:
            conversation.append({"from": "api", "value": str(example['api_response'])})
            if "assistant_response" in example and example['assistant_response'] is not None:
                conversation.append({"from": "gpt", "value": example['assistant_response']})
        return conversation

    texts = [tokenizer.apply_chat_template(format_conversation(example), tokenize=False, add_generation_prompt=True) for example in batch["conversations"]]
    return {"text": texts}

# Apply formatting
train_dataset = train_dataset.map(lambda batch: format_combined_prompts(batch, tokenizer), batched=True)
val_dataset = val_dataset.map(lambda batch: format_combined_prompts(batch, tokenizer), batched=True)


Map: 100%|████████████████████| 261/261 [00:00<00:00, 19713.91 examples/s]


In [37]:
train_dataset[0]

{'conversations': {'api_response': None,
  'assistant_response': None,
  'intent_extraction': {'entities': {'city': 'San Antonio',
    'date': 'today',
    'time_of_day': 'afternoon'},
   'intent': 'current_weather'},
  'user_input': 'What is the current weather in San Antonio today afternoon?'},
 'text': '<s>'}

In [38]:
train_dataset[1]

{'conversations': {'api_response': {'date': '2024-06-17',
   'description': 'few clouds',
   'humidity': 80,
   'location': None,
   'temperature': 19.93,
   'wind_speed': 2.44},
  'assistant_response': 'The forecast for Istanbul on 2024-06-17 night is few clouds with a temperature of 19.93°C, wind speed of 2.44 meters per second, and humidity of 80%.',
  'intent_extraction': None,
  'user_input': 'What is the forecast weather in Abu Dhabi 2024-06-17?'},
 'text': '<s>'}

In [39]:
# Modify global metric lists to include validation split
train_metrics = {"step": [], "bleu": [], "rouge": [], "perplexity": []}
val_metrics = {"step": [], "bleu": [], "rouge": [], "perplexity": []}

def compute_metrics(eval_pred, split="train"):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU
    bleu = load_metric("bleu")
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # ROUGE
    rouge = load_metric("rouge")
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Perplexity
    perplexity = torch.exp(torch.tensor(eval_pred[0].mean()))

    metrics = {
        "bleu": bleu_result['bleu'],
        "rouge": rouge_result['rougeL'].mid.fmeasure,
        "perplexity": perplexity.item(),
    }

    if split == "train":
        train_metrics["step"].append(len(train_metrics["step"]) + 1)
        train_metrics["bleu"].append(metrics["bleu"])
        train_metrics["rouge"].append(metrics["rouge"])
        train_metrics["perplexity"].append(metrics["perplexity"])
    else:
        val_metrics["step"].append(len(val_metrics["step"]) + 1)
        val_metrics["bleu"].append(metrics["bleu"])
        val_metrics["rouge"].append(metrics["rouge"])
        val_metrics["perplexity"].append(metrics["perplexity"])

    return metrics

In [40]:
# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    max_steps=30,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs_combined",
    logging_dir='./logs',
    report_to="tensorboard",
)

In [41]:
# train_metrics

In [42]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
    compute_metrics=lambda eval_pred: compute_metrics(eval_pred, split="train"),
)

Map (num_proc=2): 100%|████████| 261/261 [00:00<00:00, 1285.45 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [43]:
# trainer = SFTTrainer(
#     model=accelerator.prepare(model),
#     tokenizer=tokenizer,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     dataset_num_proc=2,
#     packing=False,
#     args=training_args,
#     compute_metrics=lambda eval_pred: compute_metrics(eval_pred, split="train"),
# )

In [44]:
# Add TensorBoard callback
trainer.add_callback(TensorBoardCallback)

You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
NotebookProgressCallback


In [45]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 30
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


In [46]:
#torch.cuda.empty()

In [47]:
torch.cuda.empty_cache()

In [48]:
# trainer.evaluate()

In [49]:
# val_results = trainer.evaluate(eval_dataset=val_dataset, metric_key_prefix="val")

In [50]:
model.push_to_hub("VatsalPatel18/phi3-mini-WeatherBot", token=os.getenv("HUGGINGFACE_HUB_TOKEN"))

adapter_model.safetensors: 100%|███████| 120M/120M [00:43<00:00, 2.77MB/s]


Saved model to https://huggingface.co/VatsalPatel18/phi3-mini-WeatherBot


In [51]:
model.save_pretrained("../models/trained")
tokenizer.save_pretrained("../models/trained")

('../models/trained/tokenizer_config.json',
 '../models/trained/special_tokens_map.json',
 '../models/trained/tokenizer.model',
 '../models/trained/added_tokens.json',
 '../models/trained/tokenizer.json')

In [52]:
# !pip install matplotlib

In [53]:
model.save_pretrained_gguf("../models/gguf/", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.62 out of 14.96 RAM for saving.


100%|█████████████████████████████████████| 32/32 [00:09<00:00,  3.38it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: ##### The current model type of mistral auto adds a BOS token.
Unsloth: ##### If you're using Ollama or GGUF etc, do not add a BOS in the chat template.
Unsloth: Converting mistral model. Can use fast conversion = True.
Unsloth: Extending ../models/gguf//tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at ../models/gguf/ into f16 GGUF format.
The output location will be ./../models/gguf/-unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 4096
INFO:hf-to-gguf:gguf: embedding length = 3072
INFO:hf-to-gguf:gguf: feed forward length = 8192
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 32
INFO:hf-to-gguf:gguf: rope theta = 10000.0
INFO:hf-to-gguf:gguf: rms norm ep

Unsloth: ##### The current model type of mistral auto adds a BOS token.
Unsloth: ##### If you're using Ollama or GGUF etc, do not add a BOS in the chat template.


Unsloth: Conversion completed! Output location: ./../models/gguf/-unsloth.Q4_K_M.gguf


In [54]:
# import matplotlib.pyplot as plt
# import pandas as pd

# def plot_metrics(metrics, title, ylabel):
#     df = pd.DataFrame(metrics)
#     plt.figure(figsize=(10, 6))
#     for column in metrics:
#         if column != "step":
#             plt.plot(df["step"], df[column], label=column)
#     plt.xlabel("Step")
#     plt.ylabel(ylabel)
#     plt.title(title)
#     plt.legend()
#     plt.show()

# # Plotting training metrics
# plot_metrics(train_metrics, "Training Metrics", "Score")

# # Plotting validation metrics
# plot_metrics(val_metrics, "Validation Metrics", "Score")



In [55]:
# from unsloth import FastLanguageModel

# model = FastLanguageModel.from_pretrained("path/to/save/model")
# tokenizer = AutoTokenizer.from_pretrained("path/to/save/tokenizer")
